In [13]:
from random import uniform
import subprocess
import numpy as np
import pandas as pd

In [14]:
df = pd.DataFrame(columns=['Tc_1','Pc_1', 'w_1', 'a_1', 'b_1','Tc_2','Pc_2', 'w_2', 'a_2', 'b_2', 'kij','type'])
for i in range(1, 500):
    R = 0.08314472

    # Compuesto 1
    name1 = 'COMP A'
    Tc_1 = round(uniform(1,1200), 2)
    #Tc_1 = 345.7
    Pc_1 = round(uniform(1,400), 2)
    #Pc_1 = 185.13
    Vc_1 = round((R*Tc_1)/(3*Pc_1), 4)
    w_1 = round(uniform(0,1), 4)
    #w_1 = 0.1090

    a_1 = round(0.42748*R**2*Tc_1**2/Pc_1, 6)
    b_1 = round(0.08664*R*Tc_1/Pc_1, 6)
    m_1 = round(0.480 + 1.574*w_1 - 0.176*w_1**2, 6)

    # Compuesto 2 
    name2 = 'COMP B'
    Tc_2 = round(uniform(1,1200), 2)
    #Tc_2 = 703.43
    Pc_2 = round(uniform(1,400), 2)
    #Pc_2 = 27.38
    Vc_2 = round((R*Tc_2)/(3*Pc_2), 4)
    w_2 = round(uniform(0,1), 4)
    #w_2 = 0.172170

    a_2 = round(0.42748*R**2*Tc_2**2/Pc_2, 6)
    b_2 = round(0.08664*R*Tc_2/Pc_2, 6)
    m_2 = round(0.480 + 1.574*w_2 - 0.176*w_2**2, 6)
    
    kij = 0.0
    
    # Orden de los compuestos
    CompA = [name1, Tc_1, Pc_1, w_1, Vc_1, a_1, b_1, m_1]
    CompB = [name2, Tc_2, Pc_2, w_2, Vc_2, a_2, b_2, m_2]
    n = 10
    
    if (Tc_2/Tc_1)**n * (Pc_1/Pc_2) < 1:
    #if np.sqrt(Tc_2**2 + Pc_2**2) < np.sqrt(Tc_1**2 + Pc_1**2): #Como vectores
        primero = CompB 
        segundo = CompA
    else:
        primero = CompA
        segundo = CompB
    
    
    # Escribir en GPECIN
    # Tengo que ver la forma de mejorar esta parte
    GPECIN = open("GPECIN.dat","w+")

    GPECIN.write("1\n")
    GPECIN.write("0 0\n")

    GPECIN.write("%s\n"%primero[0])
    GPECIN.write("%s  "%primero[1])
    GPECIN.write("%s  "%primero[2])
    GPECIN.write("%s  "%primero[3])
    GPECIN.write("%s\n"%primero[4])
    GPECIN.write("%s  "%primero[5])
    GPECIN.write("%s  "%primero[6])
    GPECIN.write("%s\n"%primero[7])

    GPECIN.write("%s\n"%segundo[0])
    GPECIN.write("%s  "%segundo[1])
    GPECIN.write("%s  "%segundo[2])
    GPECIN.write("%s  "%segundo[3])
    GPECIN.write("%s\n"%segundo[4])
    GPECIN.write("%s  "%segundo[5])
    GPECIN.write("%s  "%segundo[6])
    GPECIN.write("%s\n"%segundo[7])

    GPECIN.write("%s\n"%kij)
    GPECIN.write("0.0\n")
    GPECIN.write("2000.0")

    GPECIN.close()
    
    # Ejecutar GPEC
    proc = subprocess.Popen('GPEC.exe')
    try:
        outs, errs = proc.communicate(timeout=5) # 5 segundos
    except subprocess.TimeoutExpired:
        proc.kill()
        outs, errs = proc.communicate()
    
    # Encontrar el tipo de modelo en GPECOUT
    # Esto es suponiendo que el programa siempre lo devuelva en el mismo formato 
    fraseClave = '  Type of phase behaviour predicted by the model for this system' #64 caracteres
    tipoComportamiento = 'no hay type'
    with open('GPECOUT.DAT') as nh:
        for rec in nh:
            if rec[0:64] == fraseClave:
                for rec in nh:
                    tipoComportamiento = rec
                    tipoComportamiento = int(tipoComportamiento.split()[0]) # ya que devuelve una cadena de espacios con el numero y \n
                    break

#[name1, Tc_1, Pc_1, w_1, Vc_1, a_1, b_1, m_1]
    parametros1 = primero[1:4]
    parametros2 = segundo[1:4]
    df = df.append({'Tc_1':primero[1], 'Pc_1':primero[2], 'w_1':primero[3], 'a_1':primero[5], 'b_1':primero[6],
                    'Tc_2':segundo[1], 'Pc_2':segundo[2], 'w_2':segundo[3], 'a_2':segundo[5], 'b_2':segundo[6], 
                    'kij': kij, 'type':tipoComportamiento}, ignore_index=True)
#    print(parametros1, parametros2)
#    print(tipoComportamiento)    

In [15]:
df.head()

,Tc_1,Pc_1,w_1,a_1,b_1,Tc_2,Pc_2,w_2,a_2,b_2,kij,type
0,616.51,93.06,0.2502,12.069863,0.047723,1024.04,191.67,0.0425,16.168318,0.038487,0.0,2
1,856.03,305.96,0.8412,7.077803,0.020155,1090.18,75.58,0.3409,46.470215,0.103907,0.0,2
2,60.26,51.48,0.5700,0.208451,0.008432,1191.71,366.01,0.6880,11.466566,0.023455,0.0,no hay type
3,721.69,366.50,0.1791,4.199645,0.014185,1024.38,268.04,0.0957,11.569317,0.027531,0.0,2
4,49.98,218.26,0.8778,0.033822,0.001650,72.52,32.45,0.0825,0.478945,0.016099,0.0,2


In [16]:
df.to_csv('ceroUnokij8.csv')